# **Dicom Oluşturma**

In [2]:
import sys
import os
import itk
import vtk

# Girdi ve çıktı dizinlerini doğrudan değişkenler olarak tanımlama
dicom_directory = "Hasta_Veriler"
output_image = "output_image.nrrd"
series_name = None  # Serinin adı varsayılan olarak None

print("Başlangıç: DICOM dizininden görüntü okuma ve yazma")

# Varsayılan olarak geçerli dizin
dirName = dicom_directory if dicom_directory else "."
print(f"Kullanılan DICOM dizini: {dirName}")

PixelType = itk.ctype("signed short")
Dimension = 3

ImageType = itk.Image[PixelType, Dimension]

# DICOM dosya adlarını almak için GDCMSeriesFileNames kullanımı
namesGenerator = itk.GDCMSeriesFileNames.New()
namesGenerator.SetUseSeriesDetails(True)
namesGenerator.AddSeriesRestriction("0008|0021")
namesGenerator.SetGlobalWarningDisplay(False)
namesGenerator.SetDirectory(dirName)

print("DICOM dosyaları aranıyor...")
seriesUID = namesGenerator.GetSeriesUIDs()

if len(seriesUID) < 1:
    print("No DICOMs in: " + dirName)
    sys.exit(1)

print("The directory: " + dirName)
print("Contains the following DICOM Series: ")
for uid in seriesUID:
    print(uid)

seriesFound = False
for uid in seriesUID:
    seriesIdentifier = uid
    if series_name:
        seriesIdentifier = series_name
        seriesFound = True
    print("Reading: " + seriesIdentifier)
    fileNames = namesGenerator.GetFileNames(seriesIdentifier)

    # DICOM serisini okuma
    print(f"{seriesIdentifier} serisi okunuyor...")
    reader = itk.ImageSeriesReader[ImageType].New()
    dicomIO = itk.GDCMImageIO.New()
    reader.SetImageIO(dicomIO)
    reader.SetFileNames(fileNames)
    reader.ForceOrthogonalDirectionOff()

    # 3D görüntüyü yazma
    print("3D görüntü yazılıyor...")
    writer = itk.ImageFileWriter[ImageType].New()
    outFileName = os.path.join(dirName, seriesIdentifier + ".nrrd")
    if output_image:
        outFileName = output_image
    writer.SetFileName(outFileName)
    writer.UseCompressionOn()
    writer.SetInput(reader.GetOutput())
    print("Writing: " + outFileName)
    writer.Update()
    print("3D görüntü başarıyla yazıldı.")

    if seriesFound:
        break

print("DICOM dizininden görüntü okuma ve yazma işlemi tamamlandı.")

print("Başlangıç: 3D görüntüyü VTK ile görselleştirme")

# VTK ile 3D görüntüyü okuma ve görselleştirme
def visualize_3d_image(image_file):
    print(f"{image_file} dosyası okunuyor...")
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(image_file)
    reader.Update()

    volume_mapper = vtk.vtkGPUVolumeRayCastMapper()
    volume_mapper.SetInputConnection(reader.GetOutputPort())

    volume_property = vtk.vtkVolumeProperty()
    volume_property.ShadeOff()
    volume_property.SetInterpolationTypeToLinear()

    color_function = vtk.vtkColorTransferFunction()
    color_function.AddRGBPoint(0, 0.0, 0.0, 0.0)
    color_function.AddRGBPoint(500, 1.0, 0.5, 0.3)
    color_function.AddRGBPoint(1000, 1.0, 0.5, 0.3)
    color_function.AddRGBPoint(1150, 1.0, 1.0, 0.9)
    volume_property.SetColor(color_function)

    opacity_function = vtk.vtkPiecewiseFunction()
    opacity_function.AddPoint(0, 0.00)
    opacity_function.AddPoint(500, 0.15)
    opacity_function.AddPoint(1000, 0.15)
    opacity_function.AddPoint(1150, 0.85)
    volume_property.SetScalarOpacity(opacity_function)

    volume = vtk.vtkVolume()
    volume.SetMapper(volume_mapper)
    volume.SetProperty(volume_property)

    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume)
    renderer.SetBackground(0.1, 0.2, 0.3)
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)

    print("3D görüntü render ediliyor...")
    render_window.Render()
    render_interactor.Start()
    print("3D görüntü başarıyla görselleştirildi.")

# Girdi görüntü dosyasını değişken olarak tanımlama
image_file = "output_image.nrrd"

# 3D görüntüyü görselleştirme
visualize_3d_image(image_file)

print("3D görüntüyü VTK ile görselleştirme işlemi tamamlandı.")


Başlangıç: DICOM dizininden görüntü okuma ve yazma
Kullanılan DICOM dizini: Hasta_Veriler


KeyboardInterrupt: 

# **Dicom görselleştirme**

In [3]:
import vtk

# VTK ile 3D görüntüyü okuma ve görselleştirme
def visualize_3d_image(image_file):
    print(f"{image_file} dosyası okunuyor...")
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(image_file)
    reader.Update()

    volume_mapper = vtk.vtkGPUVolumeRayCastMapper()
    volume_mapper.SetInputConnection(reader.GetOutputPort())

    volume_property = vtk.vtkVolumeProperty()
    volume_property.ShadeOff()
    volume_property.SetInterpolationTypeToLinear()

    color_function = vtk.vtkColorTransferFunction()
    color_function.AddRGBPoint(0, 0.0, 0.0, 0.0)
    color_function.AddRGBPoint(500, 1.0, 0.5, 0.3)
    color_function.AddRGBPoint(1000, 1.0, 0.5, 0.3)
    color_function.AddRGBPoint(1150, 1.0, 1.0, 0.9)
    volume_property.SetColor(color_function)

    opacity_function = vtk.vtkPiecewiseFunction()
    opacity_function.AddPoint(0, 0.00)
    opacity_function.AddPoint(500, 0.15)
    opacity_function.AddPoint(1000, 0.15)
    opacity_function.AddPoint(1150, 0.85)
    volume_property.SetScalarOpacity(opacity_function)

    volume = vtk.vtkVolume()
    volume.SetMapper(volume_mapper)
    volume.SetProperty(volume_property)

    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume)
    renderer.SetBackground(0.1, 0.2, 0.3)
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)

    print("3D görüntü render ediliyor...")
    render_window.Render()
    render_interactor.Start()
    print("3D görüntü başarıyla görselleştirildi.")

# Girdi görüntü dosyasını değişken olarak tanımlama
image_file = "output_image.nrrd"

# 3D görüntüyü görselleştirme
visualize_3d_image(image_file)


output_image.nrrd dosyası okunuyor...
3D görüntü render ediliyor...
3D görüntü başarıyla görselleştirildi.


# **Gürültü Azaltma**

In [13]:
import vtk

# VTK ile 3D görüntüyü okuma ve görselleştirme
def visualize_3d_image(image_file):
    print(f"{image_file} dosyası okunuyor...")
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(image_file)
    reader.Update()

    volume_mapper = vtk.vtkGPUVolumeRayCastMapper()
    volume_mapper.SetInputConnection(reader.GetOutputPort())

    volume_property = vtk.vtkVolumeProperty()
    volume_property.ShadeOff()
    volume_property.SetInterpolationTypeToLinear()

    # Renk transfer fonksiyonu
    color_function = vtk.vtkColorTransferFunction()
    color_function.AddRGBPoint(0, 0.0, 0.0, 0.0)
    color_function.AddRGBPoint(500, 1.0, 0.5, 0.3)
    color_function.AddRGBPoint(1000, 1.0, 0.5, 0.3)
    color_function.AddRGBPoint(1150, 1.0, 1.0, 0.9)
    volume_property.SetColor(color_function)

    # Opaklık transfer fonksiyonu
    opacity_function = vtk.vtkPiecewiseFunction()
    opacity_function.AddPoint(0, 0.00)
    opacity_function.AddPoint(500, 0.15)
    opacity_function.AddPoint(1000, 0.15)
    opacity_function.AddPoint(1150, 0.85)
    volume_property.SetScalarOpacity(opacity_function)

    # Parazitleri azaltmak için ek ayarlar
    volume_mapper.SetBlendModeToComposite()
    volume_property.SetShade(True)
    volume_property.SetAmbient(0.1)
    volume_property.SetDiffuse(0.9)
    volume_property.SetSpecular(0.2)
    volume_property.SetSpecularPower(10.0)

    volume = vtk.vtkVolume()
    volume.SetMapper(volume_mapper)
    volume.SetProperty(volume_property)

    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume)
    renderer.SetBackground(0.1, 0.2, 0.3)
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)

    print("3D görüntü render ediliyor...")
    render_window.Render()
    render_interactor.Start()
    print("3D görüntü başarıyla görselleştirildi.")

# Gürültü azaltılmış 3D görüntüyü görselleştirme
denoised_image_file = "denoised_output_image.nrrd"
visualize_3d_image(denoised_image_file)

print("3D görüntüyü VTK ile görselleştirme işlemi tamamlandı.")


# **X-Ray'e çevirme**

In [12]:
import vtk

# VTK ile 3D görüntüyü okuma ve X-ray benzeri görselleştirme
def visualize_3d_image_as_xray(image_file):
    print(f"{image_file} dosyası okunuyor...")
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(image_file)
    reader.Update()

    # MIP (Maximum Intensity Projection) modunu ayarla
    volume_mapper = vtk.vtkGPUVolumeRayCastMapper()
    volume_mapper.SetInputConnection(reader.GetOutputPort())
    volume_mapper.SetBlendModeToMaximumIntensity()  # MIP modu

    volume_property = vtk.vtkVolumeProperty()
    volume_property.ShadeOff()
    volume_property.SetInterpolationTypeToLinear()

    # Renk transfer fonksiyonu (X-ray efekti için gri tonlama)
    color_function = vtk.vtkColorTransferFunction()
    color_function.AddRGBPoint(0, 0.0, 0.0, 0.0)    # Siyah
    color_function.AddRGBPoint(200, 0.2, 0.2, 0.2)  # Koyu gri
    color_function.AddRGBPoint(500, 0.8, 0.8, 0.8)  # Açık gri
    color_function.AddRGBPoint(1000, 1.0, 1.0, 1.0)  # Beyaz
    volume_property.SetColor(color_function)

    # Opaklık transfer fonksiyonu (X-ray efekti için)
    opacity_function = vtk.vtkPiecewiseFunction()
    opacity_function.AddPoint(0, 0.0)
    opacity_function.AddPoint(100, 0.0)
    opacity_function.AddPoint(200, 0.1)
    opacity_function.AddPoint(500, 0.2)
    opacity_function.AddPoint(1000, 0.8)
    opacity_function.AddPoint(1500, 1.0)
    volume_property.SetScalarOpacity(opacity_function)

    volume = vtk.vtkVolume()
    volume.SetMapper(volume_mapper)
    volume.SetProperty(volume_property)

    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume)
    renderer.SetBackground(0.0, 0.0, 0.0)  # Siyah arka plan
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)

    print("3D görüntü render ediliyor...")
    render_window.Render()
    render_interactor.Start()
    print("3D görüntü başarıyla görselleştirildi.")

# Girdi görüntü dosyasını değişken olarak tanımlama
image_file = "denoised_output_image.nrrd"

# 3D görüntüyü X-ray benzeri şekilde görselleştirme
visualize_3d_image_as_xray(image_file)

print("3D görüntüyü VTK ile X-ray benzeri görselleştirme işlemi tamamlandı.")


denoised_output_image.nrrd dosyası okunuyor...
3D görüntü render ediliyor...
3D görüntü başarıyla görselleştirildi.
3D görüntüyü VTK ile binary görselleştirme işlemi tamamlandı.
